In [21]:
from langgraph.graph  import StateGraph,START,END
from langchain_ollama import ChatOllama
from typing import TypedDict,Annotated
from pydantic import BaseModel, Field
import operator

In [22]:
model = ChatOllama(model="tinyllama")

In [23]:
class EvaluationResult(BaseModel):
    feedback: str = Field(description="Detailed feedback on the essay.")
    score : int = Field(description="Score out of 10 for the essay quality.", ge=0, le=10)

In [24]:
structured_model = model.with_structured_output(EvaluationResult)

In [25]:
essay = """Artificial Intelligence (AI) is one of the most transformative technologies of the 21st century. It is reshaping industries, improving efficiencies, and changing the way humans interact with technology. In Pakistan, AI is gradually emerging as a critical tool that can help drive economic growth, improve healthcare, enhance education, and strengthen governance. Despite challenges, the potential of AI in Pakistan is immense, and it is beginning to play a significant role in various sectors.

AI in Healthcare

One of the most important applications of AI in Pakistan is in the healthcare sector. AI-powered tools can help diagnose diseases faster and more accurately than traditional methods. For example, AI algorithms can analyze medical images to detect cancers or other diseases at an early stage. In rural areas of Pakistan, where access to medical specialists is limited, AI-driven telemedicine solutions can provide essential health services remotely. During the COVID-19 pandemic, AI was used in Pakistan to track the spread of the virus, predict hotspots, and optimize the distribution of vaccines and medical resources.

AI in Education

Education is another area where AI is making a difference in Pakistan. AI-based learning platforms can provide personalized learning experiences for students. For instance, adaptive learning systems can identify a student’s strengths and weaknesses and suggest study material accordingly. This approach is especially useful in large classrooms where teachers cannot focus on each student individually. Moreover, AI can help in creating online content in Urdu and regional languages, making education more accessible to a wider population.

AI in Agriculture

Agriculture is a key sector of Pakistan’s economy, employing a large portion of the population. AI can help farmers increase crop yields, reduce costs, and minimize the risk of crop failure. Smart farming technologies, such as AI-powered drones and sensors, can monitor soil health, weather conditions, and crop growth in real-time. Predictive AI models can help farmers decide the best time to plant and harvest crops, reducing losses due to pests or climate challenges. By adopting AI, Pakistan’s agricultural sector can become more efficient, sustainable, and competitive in global markets.

AI in Governance and Public Services

AI can also improve governance and public services in Pakistan. For example, AI can analyze large amounts of data to identify patterns in crime, traffic management, and public health, helping government agencies make better decisions. Chatbots and AI-powered customer service systems can assist citizens in accessing government services more efficiently. AI can also be used in disaster management to predict natural calamities like floods and earthquakes, enabling timely evacuation and relief efforts.

Challenges and the Way Forward

Despite the benefits, Pakistan faces several challenges in adopting AI fully. Lack of awareness, limited infrastructure, inadequate technical expertise, and ethical concerns are major obstacles. Moreover, investment in AI research and development is still limited compared to developed countries.

To overcome these challenges, Pakistan needs to focus on building AI skills among students and professionals, improving digital infrastructure, and encouraging AI research in universities and tech startups. The government should also create policies to regulate AI, ensuring that it is used ethically and responsibly.

Conclusion

Artificial Intelligence has the potential to transform Pakistan in multiple sectors including healthcare, education, agriculture, and governance. While challenges exist, the country can harness AI to drive economic growth, improve quality of life, and address social issues effectively. With proper investment, education, and regulation, AI can become a powerful tool for Pakistan’s sustainable development and global competitiveness."""

In [ ]:
prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {essay}'
structured_model.invoke(prompt).feedback

In [ ]:
class EssayState(TypedDict):
    
    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_feedback: Annotated[list[int], operator.add]
    avg_score: float

In [ ]:
def evaluate_language(state: EssayState) :
    prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)

    return {
        "language_feedback": output.feedback,
        "individual_feedback": [output.score]
    }

In [ ]:
def evaluate_analysis(state: EssayState) :
    prompt = f'Evaluate the analytical depth of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)

    return {
        "analysis_feedback": output.feedback,
        "individual_feedback": [output.score]
    }

In [ ]:
def evaluate_thought(state: EssayState) :
    prompt = f'Evaluate the clarity of thought in the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)

    return {
        "clarity_feedback": output.feedback,
        "individual_feedback": [output.score]
    }

In [ ]:
def final_evaluation(state: EssayState) :
   prompt = f'Based on the following feedbacks create a summarized feedback \n language feedback - {state["language_feedback"]} \n depth of analysis feedback - {state["analysis_feedback"]} \n clarity of thought feedback - {state["clarity_feedback"]}'
   overallfeedback = model.invoke(prompt).content

   avg_score = sum(state["individual_feedback"])/len(state["individual_feedback"]) 


   return {"overall_feedback": overallfeedback,
       "avg_score": avg_score}

In [ ]:
graph = StateGraph(EssayState)


graph.add_node('evaluate_language',evaluate_language)
graph.add_node('evaluate_analysis',evaluate_analysis)
graph.add_node('evaluate_thought',evaluate_thought)
graph.add_node('final_evaluation',final_evaluation)


graph.add_edge(START,'evaluate_language')
graph.add_edge(START,'evaluate_analysis')
graph.add_edge(START,'evaluate_thought')

graph.add_edge('evaluate_language','final_evaluation')
graph.add_edge('evaluate_analysis','final_evaluation')  
graph.add_edge('evaluate_thought','final_evaluation')
graph.add_edge('final_evaluation',END)

workflow = graph.compile()

In [ ]:
initial_state: EssayState = {
    "essay": essay,
}


workflow.invoke(initial_state)  

In [ ]:
workflow